<a href="https://colab.research.google.com/github/HashimHilal-QUT/LLM/blob/main/IFN581_Quiz_Bank_from_Text_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INTRO

In [ ]:
# @title 1. Setup Environment
!pip install python-docx
from google.colab import ai
from docx import Document
import requests
import io
import base64
from IPython.display import clear_output

print("✅ Environment ready.")

✅ Environment ready.


In [ ]:
# @title 2. Initialize Secure Data Connection
# Source API Key
_m = "aHR0cHM6Ly9naXRodWIuY29tL0hhc2hpbUhpbGFsLVFVVC9MTE0vcmF3L21haW4vSUZOODUxX0FJX1F1aXpNYXN0ZXIvRGF0YXNldC9RdWl6TWFzdGVyLmRvY3g="

try:

    _u = base64.b64decode(_m).decode('utf-8')
    response = requests.get(_u)
    response.raise_for_status()

    doc = Document(io.BytesIO(response.content))
    question_bank = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

    # Clear the execution output so no URL-related logs remain visible
    clear_output()
    print("✅ Question Bank loaded ")

except Exception as e:
    clear_output()
    print("❌ Connection Error. Please check your internet or repository access.")

✅ Question Bank loaded 


In [ ]:
# @title 3. Start Interactive Quiz
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# 1. Custom CSS to force larger font in text boxes and buttons
display(HTML("""
<style>
    .widget-text input { font-size: 16px !important; line-height: 1.5 !important; }
    .widget-output { font-size: 16px !important; }
    .widget-button { font-weight: bold !important; }
</style>
"""))

# Initialize conversation state
state = {
    'transcript': f"""SYSTEM: You are a Strict Quiz Master.
    SOURCE MATERIAL: {question_bank}
    STRICT RULES:
    1. Only use questions from SOURCE MATERIAL.
    2. Answers are in Bold text with choices indicated by A:, B:, C: and D:
    2. Provide feedback if the answer is correct or not then move on to the next question in ONE message.
    3. No conversational filler or follow-up questions.
    4. Provide 'FINAL GRADE REPORT' at the end or on exit.
    """,
    'is_finished': False
}

# --- UI Elements with Larger Styling ---
output_area = widgets.Output(layout={
    'border': '2px solid #4A90E2',
    'padding': '15px',
    'height': '500px',
    'overflow_y': 'scroll',
    'margin': '0 0 10px 0'
})

text_input = widgets.Text(
    placeholder='Type your answer here...',
    layout={'width': '70%', 'height': '45px'}
)

send_button = widgets.Button(
    description='Send Answer',
    button_style='primary',
    layout={'width': '15%', 'height': '45px'}
)

exit_button = widgets.Button(
    description='Exit & Grade',
    button_style='danger',
    layout={'width': '15%', 'height': '45px'}
)

def get_ai_response(user_text):
    state['transcript'] += f"\nStudent: {user_text}\nAI:"
    raw_response = ai.generate_text(state['transcript'])
    state['transcript'] += f" {raw_response}"
    return raw_response.replace("**", "")

def handle_send(b):
    if state['is_finished']: return
    user_val = text_input.value.strip()
    if not user_val: return

    with output_area:
        print(f"➜ You: {user_val}")
        text_input.value = ""
        response = get_ai_response(user_val)
        print(f"🤖 AI: {response}\n" + "━"*40)

        if "FINAL GRADE REPORT" in response:
            state['is_finished'] = True
            text_input.disabled = True

def handle_exit(b):
    if state['is_finished']: return
    with output_area:
        print("\n🛑 Ending Quiz Early...")
        response = get_ai_response("The student clicked EXIT. Stop the quiz and provide the FINAL GRADE REPORT now.")
        print(f"🤖 AI: {response}")
        state['is_finished'] = True
        text_input.disabled = True

send_button.on_click(handle_send)
text_input.on_submit(handle_send)
exit_button.on_click(handle_exit)

# --- Launch UI ---
display(output_area)
display(widgets.HBox([text_input, send_button, exit_button]))

with output_area:
    first_msg = get_ai_response("Start the quiz now by asking Question 1 from the bank.")
    print(f"🤖 AI: {first_msg}\n" + "━"*80)

Output(layout=Layout(border='2px solid #4A90E2', height='500px', margin='0 0 10px 0', overflow_y='scroll', pad…